# Лекция 2

## Файловые системы

>**ext** \- 1992
>
>**ext2** \- 2001
>
>**ext3** \- 2002
>
>**ext4** \- 2006-2008

- Журналируемые (сначала запись идет в журнал, затем на диск, если операция не проведена успешна, попытка повторяется)
> Операция записи может быть прервана например при физической неисправности или отключении устройства
- Нежурналируемые

Отсутствие файловых систем сначала было преимуществом, так как позволяло экономить ресурсы

_Блоком_ называется группа объединенных между собой секторов. Между блоками записывается _суперблок_ (или несколько, в зависимости от жд).

Помимо блоков в файловой системе хранятся блоки хранения метаданных _i-node_.
Базовая карта индексных дескрипторов хранится в суперблоке. Там же размещается таблица индексных дескрипторов, а после нееидет уже блок данных.

```
        --------------
        | суперблок  |
        --------------
        | опис.груп. |
        --------------
        |block bitmap|
        --------------
        |inode bitmap|
        --------------
        | inode table|
        --------------
        |    data    |
        --------------
```

## Файл

1. Именное обозначение (filename)
2. Индексный дескриптор/метаданные
3. Блоки данных/данные (всегда присутствует минимум один файловый дескриптор (i-node), если его нет, блок будет перезаписан)

```
block1 \
        inode
block2 /     \
              \
               Data
              /
block3 - inode   
```

>0,3%-0,7% от общего объема жд - inode

Ext не умеют динамически увеличивать количество нод.
В случае исчерпания всех индексных дескрипторов возможности создания файла/директории/другую структуру не получится.
Индексные дескрипторы привязаны к области, которая размечена в фс и в ее рамках они уникальны. Если в системе имеется несколько жд (или один жд разбит на несколько разделов), каждая из ext может содержать ноды с одинаковыми номерами.

Из данных суперблока можно извлечь следующую информацию:
- общее число блоков и нод в фс
- размер блока в фс
- число свободных нод или блоков в текущей фс
- размер индексного дескриптора
- идентификатор фс
- дата последней проверки фс
- количество произведенных монтирований
- форк состояния фс


### Монтирование

_Монтирование_ \- это закрепление размеченной части жд за определенной точкой в фхс

Может быть выполнено только в структуру, представляющую из себя каталог. Каталог в своем сегменте данных и/или инлайновом сегменте метаданных хранит информацию о других нодах, размещенных внутри данной директории.

Другие фс напр. zfs

ext3 не нуждается в дефрагментации
ext4 в два раза больше и эффективнее, поскольку является логичным эволюционным витком, ведь именно с ней происходит переход на 64-битные фс

```bash
mount #выводит список примонтированных устройств с точками монтирования
umount sdb1 #/dev/sdb1
hardlink #при жесткой линковке файлов дополнительные дескрипторы не создаются
chown
stat file #показать владельцев
chmod 770 file
```
0. бит липкости (sticky)
1. чтение
2. запись
3. выполнение

# Лекция 3

Система не работает на прямую с идентификатором пользователя в буквенном формате, он идентифицирует его числом (UID /etc/passwd). Аналогичная ситуация происходит с группой. Для ее обозначения используется групповой идентификатор GID (/etc/group). У каждого пользователя в системе есть группа. Группа, которая используется по умолчанию в виде идентификатора прописана сразу за идентификатором пользователя. Группа в конфиге может быть создана пустой (без пользователей). 

### Sticky bit

*sticky uid - SUID* 100 [000 000 000]

*SGID* 010 [000 000 000]

или 
```bash
chmod s+q #s+g
```

Восьмиричный sticky-bit 1000 +t может возвести только root. Удаление файлов из такого каталога возможно только владельцем файла или рутом. Остальные будут лишены права удалить файл, даже если их группа имеет доступ. Владелец каталога сможет убрать бит, но возвести обратно не сможет.

Пользователь www id 33

### Многозадачность и процессы

- [Задача об обедающих философах](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BE%D0%B1_%D0%BE%D0%B1%D0%B5%D0%B4%D0%B0%D1%8E%D1%89%D0%B8%D1%85_%D1%84%D0%B8%D0%BB%D0%BE%D1%81%D0%BE%D1%84%D0%B0%D1%85)
- [Round-robin](https://ru.wikipedia.org/wiki/Round-robin_(%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC))
- [Алгоритмы](https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B)

Приоритет: абсолютный и относительный
Все программы, исполняемые в рамках системы, управляются планировщиком ядра

`htop` - посмотреть процессы

Количество идентификаторов, которые могут быть запущены, конечно => *fork-bomb*

`init PID=1`

Вся структура процессов имеет древовидную основу. Исключения: процесс-сирота (orphaned), процесс-зомби.
Все процессы в рамках одной ОС имеют разные PID.

Контекст сигнальных вызовов signal context

- SIGKILL - смерть сродителя
- SIGTERM - смерть потомков

Процесс может быть не завершен, а остановлен. Посмотреть - `jobs`. `fg` запускает джоб с плюсиком, иначе нужно писать номер

`ps` - process snapshot, создает мгновенный снимок системы по текущим процессам

### Терминалы 

> TTY - teletype 

Консоль - подключение физическим путем.

Алиасы возможны из-за предварительной обработки входящих данных, организующих работу псевдотелетайпа.

/dev/urandom

nice, renice (если процесс запущен от рута, изменить его приоритет не получится)

## Лекция 5

```
|-------|
|0xfff..|
|-------|
| steak |
|-------|
| heap  |
|-------|
| data  |
|-------|
| code  |
|-------|
|0x000..|
|-------|

virtual memory
```
В момент запуска ПО вся информация, которая должна быть выполнена, загружается в сектор `code`
Любой процесс, исполняемый в рамках системы перед своим запуском проходит два этапа:

1. **Fork**

    Поскольку любое ПО в рамках ОС считает, что исполняется только оно, ему предоставляется сегмент памяти, для записи своих данных, кода и изменяемых значений (*heap* и *stack*). Место, в которое эти данные будут записаны, принято называть виртуальной памяти. ВП сильно разряжена (имеет большое кол-во незаполненных ячеек (нули) и соотносится с реальной памятью по правилам, которые мы рассмотрим далее). Создание нового процесса начинается с того, что от процесса родителя поступает системный вызов (*syscall*) с просьбой для ОС создать его дубликат. Система может отреагировать на fork тремя способами:
    
    - `fork < 0` - возвращает родителю код ошибки
    - `fork > 0` - возвращает PID дочернего процесса в случае успеха
    - `fork = 0` - особая ситуация, говорящая о том, что вы являетесь дочерним процессом, созданным сразу за получением результата этой операции
    
    Процесс-потомок, поскольку содержит копию основного процесса, имеет индексные дескрипторы файлов, сокеты и другие устр-ва, к которым обращался процесс-родитель
    
    Мьютекс, семафоры, почтовые ячейки (mail boxes)
    
2. **Exec**

    После того, как родительский экземпляр программы породил дочерний, дочерний экземпляр может загрузить себя на исполнение любой исполняемый ELF (он бывает трех типов - перемещаемый, разделяемый и исполняемый)
    
    - Перемещаемые файлы. Хранит инструкции и данные, которые могут быть связанными с разделяемыми файловыми объектами или исполняемыми
    - Разделяемый объектный файл. Так же как и перемещаемый файл, содержит инструкцию, которые могут быть связаны с другими перемещаемыми файлами или разделяемыми объектными файлами, в результате чего создается новый объектный файл.
    - Исполняемый объектный файл. Содержит полный объект, позволяющий создать процесс.
    
   Исполняемый файл должен иметь в себе заголовок (elf-header), одинаковый для 32 (dword-значения) и 64 битных систем. Он:
   
   - Содержит основные хар-ки: тип, версия формата, архитектура процессора, адрес точки входа, размеры и смещения остальных частей файла
   - Всегда распологается в начале файла
   - Размер 52 байта для 32-х разрядных и 64 байта для 64-х разрядных. Несмотря на то, что формат для обоих хедеров идентичный, последний получается больше из-за наличия внутри более длинных ссылок (8 байт против 4)
   
    Таблица заголовков содержит в себе таблицы, каждая из которых описывает отдельный сегмент программы, его атрибуты, используемые операционной системой для подготовки программы к исполнению. В отличии от хедера, таблица может распологаться в любой части файла. Смещение относительно начала указывается в заголовке. Третья часть - в заголовке секции. Опциональный параметр, elf-файл будет загружен, даже если заголовки секций не будут найдены. Позволяют оптимально разместить программу компоновщику. 
    
Shared memory - сегмент памяти, информация из которого предоставляется одновременно сразу нескольким процессам (там находятся динамически-линкуемые библиотеки)  

В процессе выполнения `exec` выполняется реаллокация (отвязывание) shared-memory сегмента. После чего из elf-хедера выполняется чтение сегментов данных кода и т.д., которые переносятся в соответствующие сегменты процесса-потомка, а данные, оставшиеся там от форка родителя, удаляются. Стек и хип приводятся в первоначальный вид, после этого из шаред-мемори выполняется аллокация (присоединение) разделяемых блоков памяти. Если в шеред-мемори отсутствуют необходимые разделяемые эльф-объекты. По заключению подготовки виртуальной памяти, для процессора потомка требуется переставить точку исполнения кода на энтри-пойнт в соответствии с данными из эльф-хедера
    
- `free` 
- `readelf` - позволяет читать эльф-хедеры
- `editelf`
- `objdump` - утилита, позволяющая вывести информацию из секции


>айнода - это метаданные о файле грубо говоря. где он находится на устройстве и в фс, название, что-то такое. маленькие файлики целиком в айноду можно запихать. страничный кэш - это куски диска закэшированные в памяти

pagehash, freememory, page table(->MTU,TLB) - механизм-прослойка для выдачи памяти, где хранится карта кусков рама для процессов, page mapping(форку выдаются куски рама родителя, но он не может в них писать - если начнет, выпадет page fault и процессу дадут его собственную память)

Выделяют два типа ошибок обращения к памяти: минорный (без обращения к диску), мажорный (с диском) 

mmap() - выделяет страницу
malloc() - использует больше памяти и происходит копирование в юзерспейс

В момент освобождения рама бесшовно для процесса реальная рама может быть заменена свопом

### Lazy loading

Поскольку у нас имеется механизм, который позволяет подгружать необходимую информацию из файлового бэкенда и использовать по мере необходимости, и механизм, который позволяет не заниматься аллокацией памтяи до тех пор, пока она не понадобилась, в момент открытия индексного дескриптора файла мы можем не считывать его с жесткого диска и не переносить в оперативную память, а реализовать т.н. механизм *ленивой загрузки*, отобразив те ячейки памяти, где должен находиться файл ячейки с файловым бэкендом, но по факту всего лишь часть таких ячеек будет загружена и помещена. Ячейки будут подгружаться по мере их необходимости и вытесняться из кэша при неиспользовании.

При работе с БД такой подход может быть неудобен и требуется перегрузка всего файла в раму даже в тех случаях, когда какой-то конкретный сегмент базы не используется, на нем могут исполняться процессы индексации, сегментации и т.д.

### cgroups

Механизм, позволяющий ограничить кол-во памяти, используемой процессом. При чем отдельно можно ограничивать память с учетом свопа и без ее. В случае превышении кол-ва используемой памяти сработает механизм, называемый outofmemorykiller. Так же при помощи cgroups можно получать статистику использования различных видов памяти, при чем как по развитии группы, так и процесса.

### Сокеты

Сокет обеспечивает двустороннюю связь типа точка-точка для процессов. Используется для межсистемного взаимодействия и межпроцессорной связи. Каждый сокет представляет собой отдельную точку связи, с которой можно совместить некое имя. например, индексный дескриптор, лежащий в рамках фхс. Сокет должен иметь тип и обязательно иметь минимум один связанный с ним процесс. Все сокеты находятся в областях, именуемых доменами. Домен сокета - это абстракция, определяющая структуру адресации и набор доступных протоколов. Сокеты не могут осуществлять кросс-доменного взаимодействия. Всего существует 23 класса сокетов. Нам интересно 2 - unix-сокеты и интернет-сокеты.

Unix-сокеты обеспечивает их адресное пространство в рамках одной ОС. Unix-сокеты называются именами файлов и имеют свое расположение в структуре каталогов. 

Интернет-сокеты можно использовать для организации связи между процессами на различных системах. 

Основные типы сокетов:
1. Поточные - двухсторонний последовательный недублированный поток данных без особых границ. В домене интернета образует протокол TCP
2. Датаграммный - двусторонний поток собщений. В отличии от поточного, возможно получение данных не в том порядке, в котором они посылались. Поток недублирующийся. Для интернет-сокетов - UDP.
3. Сокет последовательных портов обеспечивает двусторонний последовательный надежный обмен датаграммами фиксированной длины.
4. Simple-socket

UNIX-сокеты ориентированы на передачу датаграмм. но может быть специфика, заложенная протоколом. Обмен в сокете происходит по след. схеме:

1. Сервер устанавливает сокет
2. Сервер биндит его(привязывает его к имени)
3. Сервер устанавливает очередь запросов входящих сообщений. 
4. Переход в режим ожидания клиента
5. Со стороны клиента инициализируется сокет и устанавливается соединение к серверу
6. Выбор одного из соединений в очереди ожидающих подключения со стороны сервера и создания потока для получения информации от клиента
7. Открытие канала передачи данных или приведение текущего канала приема данных для передачи инфы на сторону клиента


### Loop Local

Специальный интерфейс обмена данными, как если бы он проходил через сеть с одним лишь исключением - никаких данных в сторону драйверов не поступает. Данные принимаются и отправляются виртуально.

Поскольку большинство современных приложений использует ассинхронный подход к приему и передаче информации, организуется две очереди: input и output. Сетевая система на основании данных о подключении (протокол, порт и т.д.) формирует пакет и проводит его валидацию на основании iptables (разобрать дома). Если принято решение об отправке пакета, он поступает на драйвер соответствующего устройства в ядре, где модулируется в сигналы и передается по сети. Другой абонент принимает сигнал и выполняет его обратную демодуляцию в блок данных, из которых собирается пакет, который, в свою очередь, передается на валидацию и внесение изменений правилами iptables узла-приемника. Если было принято решение о передаче данных во внутрь приложения, из пакета извлекается полезная нагрузка (пейлоад), которая помещается в очередь инпута и уже приложение на той стороне может извлечь данные из очереди инпут и обработать их. Если процесс использует локалхост, то взаимодействие с драйверами и реальными устройствами не происходит. 

Цепочка аутпута позволяет сконфигурировать процесс взаимодействия приложения, работающего на хосте, с другими приложениями в сети так, что: исходящее подключение по TCP будет возможно, преобразованный канал будет поддерживаться, а удаленные подключения из сети к данной ОС будут заблокированы